#### 一. AMQP简要说明
1. exchange, binding , queue  
AMQP(Advanced Message Queuing Protocol)是一种消息协议. 简单来说, 一个消息可以被发送到`exchange`里, `exchange`相当于邮局或信箱(mailbox); 然后根据一定的规则把消息拷贝分发到相应的`queue`里面, 这里的规则叫做`binding`. 然后, `broker`把每一条消息分发给对应的客户端, 或者等客户端需要的时候自己来fetch
<img src="https://www.rabbitmq.com/img/tutorials/intro/hello-world-example-routing.png">  

2. 针对消息的通知-message acknowledgements  
    * 当一条消息分发给`consumer`后, `consumer`需要通知`broker`已收到该消息. 通知方式有2种: 自动通知和手动通知. 当启用了`message acknowledgement`后, 一旦`broker`收到针对某条消息的通知, broker将会把该条消息完全的删除.
    * 某些情况下, 一个message并不能被现有的`binding`规则找到对应的`queue`, `broker`会将该消息返还给`Publisher`, `Publisher`可以选择丢弃该消息还是将消息放入死信队列("dead letter queue")
    
#### 二. Exchange Types
`Exchange`可以决定一条消息被路由发送给0个或多个`queue`中:
* 路由的算法依赖于`Exchange`的类型
* 路由的规则成为`binding`

|Exchange类型|	Default pre-declared names|
|:------:|:-------:|
|Direct exchange	|(Empty string) and amq.direct|
|Fanout exchange	|amq.fanout|
|Topic exchange	|amq.topic|
|Headers exchange	| &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;amq.match (and amq.headers in RabbitMQ)|

1. **`Direct Exchange`**(point to point单播)   
`Direct Exchange`使用消息的`routing key`来决定吧消息分发给哪个`queue`:
    * `queue`使用`routing key`(k)和`exchange`绑定
    * 当一条`routing key`为(R)消息发送到一个`Direct Exchange`中, 当R=K时, `exchange`将该条消息拷贝转发  
    
   `Direct exchanges`通常被用于在多个worker(同一任务的多个实例)之间分发任务, 采用轮询的方式分发; 即可以在多个`consumer`之间实现负载均衡
   
2. **`Fanout Exchange`** (pub-sub多播)  
    * `fanout exchange`会把消息路由到所有与之绑定的`queue`中, 并忽略队列的`routing key`. 如果有N个`queue`绑定在了一个`fanout exchange`中, 则发送到该`exchange`的消息将会被拷贝并分发给所有的N个`queue`. 
    * 它通常用来实现消息广播.
    
3. **`Topic Exchange`**(多播)  
`Topic Exchange`是一种消息多播的exchange. 把消息的`routing key`转发给所有与之匹配的`bingding key`对应的`queue`里

4. **`Headers Exchange`**(单播)  
`Headers Exchange`是带匹配的`Direct Exchange`. 只是它不使用`routing key`与`queue`匹配, 而是使用消息的`head`可进行匹配. 绑定queue和exchange时, 也会指定一组`header`. 当消息的`header`和queue的`header`相同时消息就会被转发到对应的多个queue内. 匹配的方式分为:
    * all: 所有header相等时才转发给这个queue 
    * any: 满足任意一个header时才转发给这个queue
    
    
### 三. Queue
Queue除了一些和绑定exchange相关的属性外, 还包含一些自己的属性: 
* Name
* Durable (broker重启后, queue是否幸存)
* Exclusive (只允许一个连接,  当连接关闭后queue被删除)
* Auto-delete (queue至少有1个consumer, 否则自动删除自己)
* Arguments (optional; used by plugins and broker-specific features such as message TTL, queue length limit, etc)


#### 四. Consumers
Consumers消息接收分为2种方式
* Have messages delivered to them ("push API")
* Fetch messages as needed ("pull API")